In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

df_od_matrix = pd.read_csv("C:\\Users\\류석현\\Desktop\\미래기술첼린지\\2023 미래기술챌린지 과제가이드 (과제3 실시간 주문 대응 Routing 최적화) v4\\과제3 실시간 주문 대응 Routing 최적화 (od_matrix) 수정완료.csv", encoding="cp949")
df_veh_table = pd.read_csv("C:\\Users\\류석현\\Desktop\\미래기술첼린지\\2023 미래기술챌린지 과제가이드 (과제3 실시간 주문 대응 Routing 최적화) v4\\veh_table.csv", encoding="cp949")
df_veh_table2 = pd.read_csv("C:\\Users\\류석현\\Desktop\\미래기술첼린지\\2023 미래기술챌린지 과제가이드 (과제3 실시간 주문 대응 Routing 최적화) v4\\veh_table.csv", encoding="cp949")
df_orders_table2 = pd.read_csv("C:\\Users\\류석현\\Desktop\\미래기술첼린지\\2023 미래기술챌린지 과제가이드 (과제3 실시간 주문 대응 Routing 최적화) v4\\바뀐주문.csv", encoding ="cp949")
df_Terminals = pd.read_csv("C:\\Users\\류석현\\Desktop\\미래기술첼린지\\2023 미래기술챌린지 과제가이드 (과제3 실시간 주문 대응 Routing 최적화) v4\\과제3 실시간 주문 대응 Routing 최적화 (Terminals).csv", encoding = "cp949")
df_d_to_o = pd.read_csv("C:\\Users\\류석현\\Desktop\\미래기술첼린지\\우리가 따로 작업\\d(터미널)을 가장 가까운 o(터미널)데이터.csv")


In [2]:
########df_orders_table 가공 > 각 일, 배치, o 마다 주문건수 10개이하로 맞춰서 새로 가공함 #############
######## 위에 csv 불러오는 코드에서 df_orders_table2로 불러오는 것만 수정하면 아래 다 돌아가 ####### 

day = ["2023.5.1", "2023.5.2", "2023.5.3", "2023.5.4", "2023.5.5", "2023.5.6"]

group1 = (0, 1, 2, 3)

# 칼럼 이름들과 빈 데이터 프레임 생성
columns = ["주문ID", "하차지_위도", "하차지_경도", "착지ID", "CBM","하차가능시간_시작", "하차가능시간_종료", 
           "하차작업시간(분)", "터미널ID", "date", "Group", "마감기한", "출발시간", "클러스터"]

df_orders_table = pd.DataFrame(columns=columns)


for days in day :
    for groups in group1 :
        for o_index, row in df_Terminals.iterrows():
            o = row['ID']

            COF = (df_orders_table2["date"] == days) & (df_orders_table2["Group"] == groups) & (df_orders_table2["터미널ID"] == o)
            order_curr = df_orders_table2[COF].sort_values(['착지ID', 'CBM', '마감기한'], ascending=[False, False, True])

            if (len(order_curr) >= 10) :
                order_curr = order_curr[0:10]
                df_orders_table = pd.concat([df_orders_table, order_curr], ignore_index=True)
            else :
                df_orders_table = pd.concat([df_orders_table, order_curr], ignore_index=True)

columns_to_drop = ["하차가능시간_시작", "하차가능시간_종료"]
df_orders_table = df_orders_table.drop(columns=columns_to_drop)
                
df_orders_table


,주문ID,하차지_위도,하차지_경도,착지ID,CBM,하차작업시간(분),터미널ID,date,Group,마감기한,출발시간,클러스터,Unnamed: 0,하차가능시간_종료_x,하차작업시간(분)_x
0,M18385491,35.577640,129.314014,D_332,7.108175,NaN,O_9,2023.5.1,0,2023.5.3 23:00,2023.5.1 0:00,11,0.0,2:00,60.0
1,M18434024,35.383357,129.019050,D_301,10.399642,NaN,O_9,2023.5.1,0,2023.5.3 23:00,2023.5.1 0:00,1,1.0,6:00,60.0
2,M18434020,35.383357,129.019050,D_301,7.183155,NaN,O_9,2023.5.1,0,2023.5.3 23:00,2023.5.1 0:00,1,2.0,6:00,60.0
3,M18386998,35.845893,128.762211,D_293,31.726262,NaN,O_9,2023.5.1,0,2023.5.3 23:00,2023.5.1 0:00,6,3.0,15:00,60.0
4,M18386900,35.845893,128.762211,D_293,24.709946,NaN,O_9,2023.5.1,0,2023.5.3 23:00,2023.5.1 0:00,6,4.0,15:00,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4252,M18394323,37.019091,127.255364,D_414,33.498993,NaN,O_673,2023.5.6,3,2023.5.9 17:00,2023.5.6 18:00,9,4274.0,18:00,60.0
4253,M18393981,35.881646,128.841459,D_409,10.271419,NaN,O_673,2023.5.6,3,2023.5.9 17:00,2023.5.6 18:00,6,4275.0,17:00,60.0
4254,M18393365,35.154450,126.782867,D_385,32.230710,NaN,O_673,2023.5.6,3,2023.5.9 17:00,2023.5.6 18:00,10,4276.0,6:00,60.0
4255,M18394271,36.709208,127.593821,D_375,18.154214,NaN,O_673,2023.5.6,3,2023.5.9 17:00,2023.5.6 18:00,3,4277.0,15:00,60.0


In [3]:
df_veh_table['Used'] = 0

# date 칼럼을 pandas datetime으로 변환
df_orders_table['date'] = pd.to_datetime(df_orders_table['date'])

df_veh_table['현재시각'] = '2023-05-01 00:00:00'
df_veh_table['현재시각'] = pd.to_datetime(df_veh_table['현재시각'])

df_orders_table['d_near_origin'] = None

for _, row in df_d_to_o.iterrows():
    var = (df_orders_table['착지ID'] == row['dterminal'])
    df_orders_table.loc[var, 'd_near_origin'] = row['o']


def time_minute(origin, destination, df_odmatrix):
    try:
        # Origin, Destination 에따른 해당 인덱스를 true, false로 나타내는데, 한인덱스만 나올거
        n = ((df_odmatrix["origin"] == origin) & (df_odmatrix["Destination"] == destination))
        # true, false만 나타나므로 그걸 이용해서 true인 값을 n1에 저장
        n1 = df_odmatrix[n]
        # dc = 거리
        dc = n1.iloc[0, 2]
        # tm = time_minute 걸리는 시간
        tm = n1.iloc[0, 3]
        return tm

    # 값이 안나오고 오류가 나올시 아래구문 탐
    except KeyError:
        print("해당 터미널 간의 거리 정보를 찾을 수 없습니다.")
        return None

    # travel_time 계산해주는 함수


def travel_time(o, once_route, nearest_origin):
    global df_od_matrix

    veh_time = 0
    keep = []
    keep.append(o)
    keep.extend(once_route)

    # 배달을 끝난후 차량의 시각을 저장.
    for i in range(2, len(keep) + 1):
        tm = time_minute(keep[i - 2], keep[i - 1], df_od_matrix)
        veh_time += tm
        if (i == len(keep)):
            veh_time = veh_time + time_minute(keep[i - 1], nearest_origin, df_od_matrix)

    return veh_time


# result_routes의 마지막 주문의 착지 ID를 확인해서 해당 터미널의 베이스캠프를 뱉어내는 함수

def update_result_routes(current_route):
    global df_orders_table
    # 처리된 주문을 기존 데이터프레임에서 삭제
    for i in range(len(current_route)):
        df_orders_table = df_orders_table[df_orders_table['주문ID'] != current_route[i]]



In [4]:

def process_orders_with_deadline(orders_df, veh_df, time_10):

    # 결과를 저장할 리스트
    routes = []
    total_cost = 0
    max_cap_veh = set()
    processed_orders = set()
    slack = 5  # 나중에 최적화
#     slack = 5  # 나중에 최적화

    no_time_dest = set()

    # 각 차량에 대해 처리
    for veh_index, veh_row in veh_df.iterrows():
        veh_num = veh_row['VehNum']
        max_capacity = veh_row['MaxCapaCBM']
        start_center = veh_row['StartCenter']
        variable_cost = veh_row['VariableCost']
        
        fixed_cost = veh_row['FixedCost']

        # 이미 처리된 라우트 건너뛰기
        if veh_num in max_cap_veh:
            continue

        current_capacity = 0  # 현재 차량의 용량
        current_location = start_center  # 현재 차량의 위치
        current_route = []  # 현재 차량의 경로
        current_time = pd.to_datetime(time_10)  # 현재 시간을 주문 중 가장 빠른 출발시간으로 초기화
        once_route = []
        time_log = [current_time]
        order_travel_distance = 0

        # 주문 처리
        for index, order in orders_df.iterrows():
            order_id = order['주문ID']
            order_cbm = order['CBM']
            order_destination = order['착지ID']

            # 라우트가 비어 있으면 목적지 초기화
            if not current_route:
                route_destination = order_destination
                

            # 못보내는 주문 건너뛰기
            if order_destination in no_time_dest:
                continue

            # 이미 처리된 주문은 건너뛰기
            if order_id in processed_orders:
                continue

            # 다른 목적지는 건너 뛰기
            if route_destination != order_destination:
                continue

            # 도착지 터미널이 od_matrix_df에 있는지 확인
            filtering = (df_od_matrix["Destination"] == order_destination) & (df_od_matrix["origin"] == current_location)
            order_travel_time = float(
            df_od_matrix[filtering]["Time_minute"].iloc[0])  # pd.to_timedelta 로 인해서 데이터프레임 형식으로 나감...
            order_travel_distance = float(df_od_matrix[filtering]["Distance_km"].iloc[0])

            # 주문이 처리 가능한지 확인(CBM 상한을 안 넘는지)
            if current_capacity + order_cbm <= max_capacity:
                # 주문 처리
                current_capacity += order_cbm
                current_route.append(order_id)
                current_time = pd.to_datetime(time_10) + pd.to_timedelta(order_travel_time, unit='m')
                
                # 현재 시간, 하차 후의 시간 저장
#                 arrival_time = str(current_time)[11:]
#                 end_time = str(current_time + pd.to_timedelta(240, unit='m'))[11:]
                
                once_route.append(order_destination)
                time_log.append(current_time)
        
        #############3꽉채워가기 코드 추가부분##############
        #현재 차량의 경로와 발생 비용을 결과 리스트에 추가, 적절한 배차 상태에 있는지 확인
        if current_route:
            # 차량이 max 상태인지 확인
            if current_capacity <  max_capacity - slack:
                fit_cbm = min([x for x in veh_cbm if x >= current_capacity])
                for veh_index, veh_row in veh_df[veh_df['MaxCapaCBM'] == fit_cbm].iterrows():
                    veh_num = veh_row['VehNum']
                    fixed_cost = veh_row['FixedCost']
                    variable_cost = veh_row['VariableCost']
                    # 이미 처리된 라우트 건너뛰기
                    if veh_num in max_cap_veh:
                        continue
                    else:
                        break


        # 현재 차량의 경로와 발생 비용을 결과 리스트에 추가
        if once_route:
            # 복귀지점 반환받고, 처리된 주문 삭제 함수
            nearest_origin = start_center

            # 시작 - 하차 - 복귀 까지 travel_time 구하는 함수
            veh_time = travel_time(o, once_route, nearest_origin)

            # time-log에 복귀 추가
            time_log.append(pd.to_datetime(time_10) + pd.to_timedelta(len(once_route) * 60 + veh_time, unit='m'))

            # 차량의 현재시각 update
            df_veh_table.loc[df_veh_table['VehNum'] == veh_num, '현재시각'] = pd.to_datetime(time_10) + pd.to_timedelta(
                len(once_route) * 60 + veh_time, unit='m')
            
            #df 업데이트
            update_result_routes(current_route)
            
            ### 2. nearest_origin으로 복귀한 차량의 travel distance
            fi = (df_od_matrix["origin"] == once_route[-1]) & (df_od_matrix["Destination"] == nearest_origin)
            order_travel_distance += df_od_matrix.loc[fi, "Distance_km"].iloc[0]

            # once_route에 nearest_origin 붙이기
            once_route.append(nearest_origin)

            # 차량의 (현재) 위치 업데이트
            df_veh_table.loc[df_veh_table['VehNum'] == veh_num, 'StartCenter'] = nearest_origin
            
            
            #차량의 사용 여부 update
            df_veh_table.loc[df_veh_table['VehNum'] == veh_num, 'Used'] = 1

            #             current_cost = fixed_cost + variable_cost * order_travel_distance
            current_cost = variable_cost * order_travel_distance
            total_cost += current_cost
            routes.append(
                (veh_num, start_center, current_route, once_route, current_cost, current_capacity, current_time,
                 order_travel_distance, fixed_cost, variable_cost, veh_time, time_log))

            max_cap_veh.add(veh_num)

        for ord in current_route:
            processed_orders.add(ord)

    return routes, total_cost

In [5]:
# 주문 처리
from tqdm import tqdm
from datetime import datetime, timedelta

day = ["2023-05-01", "2023-05-02", "2023-05-03", "2023-05-04", "2023-05-05", "2023-05-06"]
#day = ["2023-05-01", "2023-05-02", "2023-05-03"]
#day = ["2023-05-01", "2023-05-02"]

group1 = (0, 1, 2, 3)

veh_cbm = [27, 33, 42, 51, 55]

# 주문결과 테이블
col = ("VehicleID", "Sequence", "ORD_NO", "SiteCode", "ArrivalTime", "WaitingTime", "ServiceTime",
       "DepartureTime", "Delivered")
ORD_RESULT = pd.DataFrame(columns=col)

# 배차결과 테이블
col = ("VehicleID", "Count", "Volume", "TravelDistance", "WorkTime", "TravelTime", "ServiceTime", "WaitingTime",
       "TotalCost", "FixedCost", "VariableCost")
VEH_RESULT = pd.DataFrame(columns=col)

# df_orders_table 데이터프레임의 모든 칼럼 가져오기
col = df_orders_table.columns

# 새롭게 쓸df에 모든 칼럼 추가(df_orders_table의 칼럼만 담고 모든 데이터없음)
ord_curr2 = pd.DataFrame(columns=col)


pd.set_option('display.max_columns', None) ## 모든 열을 출력한다.

start_day = day[0]
start_time = datetime(year=2023, month=4, day=30, hour=18, minute=0)  # 시작 시간을 설정

#time_10 = datetime(year=2023, month=4, day=30, hour=23, minute=50)

result_routes_list=[]
result_routes_list2=[]
a = 0

# 주문 처리

progress_bar = tqdm(day, total=6, desc="Processing Orders")

for days in progress_bar :    
    for groups in group1 :
        start_time += timedelta(hours=6)  # 시간을 6시간씩 증가
        time_10 = start_time - timedelta(minutes=30)
        for i in range(12) :
            time_10 += timedelta(minutes=30)
            for o_index, row in df_Terminals.iterrows():

                o = row['ID']

                COF = (df_orders_table["date"] == days) & (df_orders_table["Group"] == groups) & (df_orders_table["터미널ID"] == o)
                order_curr = df_orders_table[COF].sort_values(['착지ID', 'CBM', '마감기한'], ascending=[False, False, True])

                CVF =  ((df_veh_table["현재시각"] <= pd.Timestamp(time_10)) & (df_veh_table["StartCenter"] == o))
                veh_curr = df_veh_table[CVF].sort_values(by=['Used','MaxCapaCBM'], ascending=[False, False])

                result_routes, result_total_cost= process_orders_with_deadline(order_curr, veh_curr, time_10)

                result_routes_list.append(result_routes)

            #     # 결과 출력
            #     print("차량 경로:")
            #     for veh_num, route, cost, capacity, time, distance, fixed_cost, variable_cost in result_routes:
            #         print(f"차량 {veh_num} 경로: {route}")
            #     print("총 발생 비용:", result_total_cost)

                Count = 0
                Volume = 0
                TravelDistance = 0
                WorkTime = 0
                TravelTime = 0
                ServiceTime = 0
                WaitingTime = 0
                TotalCost = 0
                FixedCost = 0
                VariableCost = 0


                for veh_num, sc, route, loc, cost, capacity, time, distance, fixed_cost, variable_cost, veh_time, t_log in result_routes:
                    Count = len(route)
                    Volume = capacity
                    TravelDistance = distance
                    TravelTime = veh_time
                    ServiceTime = Count * 60
                    WaitingTime = 0  # 모름, 수정
                    WorkTime = TravelTime + ServiceTime + WaitingTime
                    TotalCost = cost
                    FixedCost = fixed_cost
                    VariableCost = variable_cost * distance

                    if VEH_RESULT[VEH_RESULT["VehicleID"] == veh_num].empty:
                        new_row = [veh_num, Count, Volume, TravelDistance, WorkTime, TravelTime, ServiceTime,
                                   WaitingTime, TotalCost+FixedCost, FixedCost, VariableCost]
                        VEH_RESULT.loc[len(VEH_RESULT)] = new_row
                    else:
                        filt = VEH_RESULT['VehicleID'] == veh_num
                        VEH_RESULT.loc[filt, 'Count'] += Count
                        VEH_RESULT.loc[filt, 'Volume'] += Volume
                        VEH_RESULT.loc[filt, 'TravelDistance'] += TravelDistance
                        VEH_RESULT.loc[filt, 'WorkTime'] += WorkTime
                        VEH_RESULT.loc[filt, 'TravelTime'] += TravelTime
                        VEH_RESULT.loc[filt, 'ServiceTime'] += ServiceTime
                        VEH_RESULT.loc[filt, 'WaitingTime'] += WaitingTime
                        VEH_RESULT.loc[filt, 'TotalCost'] += TotalCost
            #                     VEH_RESULT.loc[filt, 'FixedCost'] += FixedCost
                        VEH_RESULT.loc[filt, 'FixedCost'] = FixedCost
                        VEH_RESULT.loc[filt, 'VariableCost'] += VariableCost

    progress_bar.set_postfix(Order=f"{days}/{6}")

print("차량 결과")
print(VEH_RESULT)


Processing Orders: 100%|█████████████████████████████████████████| 6/6 [1:51:02<00:00, 1110.40s/it, Order=2023-05-06/6]

차량 결과
    VehicleID Count      Volume  TravelDistance     WorkTime   TravelTime  \
0       VEH_2    19  305.900042     5321.622779  5009.498650  3869.498650   
1       VEH_3    17  265.522614     6584.812585  5680.762744  4660.762744   
2      VEH_22    15  465.785566     4558.067965  4124.191753  3224.191753   
3       VEH_4    17  363.784065     6401.469598  5661.016199  4641.016199   
4       VEH_5    16  290.135131     6541.292271  5626.511212  4666.511212   
..        ...   ...         ...             ...          ...          ...   
471   VEH_759     1   45.000000      599.388020   453.540642   393.540642   
472    VEH_66     3   59.434316      665.569849   727.900584   547.900584   
473    VEH_18     1   27.406088      214.405158   219.690762   159.690762   
474    VEH_19     1   32.906627       24.521320    95.166215    35.166215   
475   VEH_261     1   23.329154      265.466370   285.044266   225.044266   

    ServiceTime WaitingTime    TotalCost FixedCost  VariableCost  
0 

In [6]:
# 주문결과 테이블
col = ("VehicleID", "Sequence", "ORD_NO", "SiteCode", "ArrivalTime", "WaitingTime", "ServiceTime",
       "DepartureTime", "Delivered")
ORD_RESULT = pd.DataFrame(columns=col)

col2 = ("VehicleID", "Sequence", "ORD_NO", "SiteCode", "ArrivalTime", "WaitingTime", "ServiceTime",
       "DepartureTime", "Delivered")
ORD_PRED = pd.DataFrame(columns = col2)
empty_ORD = ORD_RESULT


for result_routes in result_routes_list:
    for veh_num, sc, route, loc, cost, capacity, time, distance, fixed_cost, variable_cost, veh_time, t_log in result_routes:
        sequence = 0
        empty_ORD = pd.DataFrame(columns=col)
        
        # 차량 초기 등록에만 실행하도록 수정
        if sequence == 0:
            sequence += 1
            new_row = {"VehicleID" : veh_num,
                    "Sequence" : sequence,
                    "ORD_NO" : 'Null',
                    "SiteCode" : sc, # 터미널
                    "ArrivalTime" : str(t_log[0]), 
                    "WaitingTime" : 0, # 나중에 변경
                    "ServiceTime" : 0, # 나중에 변경
                    "DepartureTime" : str(t_log[0]), 
                    "Delivered" : 'Null'}
            empty_ORD.loc[empty_ORD.shape[0]] = new_row

        for index in range(len(route)+1):
            sequence += 1
            if loc[index][0] != 'O':
                new_row = {"VehicleID" : veh_num,
                        "Sequence" : sequence,
                        "ORD_NO" : route[index],
                        "SiteCode" : loc[index], 
                        "ArrivalTime" : str(t_log[index+1]), 
                        "WaitingTime" : (index)*60,
                        "ServiceTime" : 60,
                        "DepartureTime" : str(t_log[index+1]+timedelta(minutes=(index+1) * 60)), # 나중에 변경
                        "Delivered" : 'Null'}
                empty_ORD.loc[empty_ORD.shape[0]] = new_row
            else:
                new_row = {"VehicleID" : veh_num,
                    "Sequence" : 1,
                    "ORD_NO" : 'Null',
                    "SiteCode" : loc[index], # 터미널
                    "ArrivalTime" : str(t_log[index+1]), # 나중에 변경
                    "WaitingTime" : 0, # 나중에 변경
                    "ServiceTime" : 0, # 나중에 변경
                    "DepartureTime" : None, # 나중에 변경
                    "Delivered" : 'Null'}
                empty_ORD.loc[empty_ORD.shape[0]] = new_row

        ORD_PRED = pd.concat([ORD_PRED, empty_ORD], ignore_index = True)

In [7]:
# if str(time_10)[11:] in ('0:00', '06:00', '12:00', '18:00', '24:00'):
# 시간 표시
time_10 = datetime(2023, 5, 1, 2, 0)

# 배송상태 업데이트
for _, row in ORD_PRED.iterrows():
    if row['ORD_NO'] != 'Null':
        if row['DepartureTime'] <= str(time_10):
            row['Delivered'] = 'Yes'
        else:
            row['Delivered'] = 'No'
    else:
        row['Delivered'] = 'Null'

# ORD_PRED에서 차량별로 처리 -> for 문
for veh in ORD_PRED['VehicleID'].unique():
    ex = ORD_PRED[ORD_PRED['VehicleID'] == veh]

    # DepartureTime 수정
    ex['DepartureTime'] = ex['DepartureTime'].fillna(method='bfill')
    ex = ex.drop_duplicates(subset=['SiteCode', 'DepartureTime'])

    ORD_RESULT = pd.concat([ORD_RESULT, ex], ignore_index=True)

filt = ORD_RESULT['SiteCode'].str.contains('O', case=False)
ORD_RESULT.loc[filt, 'WaitingTime'] = pd.to_datetime(ORD_RESULT.loc[filt, 'DepartureTime']) - pd.to_datetime(ORD_RESULT.loc[filt, 'ArrivalTime'])
  

C:\Users\류석현\AppData\Local\Temp\ipykernel_12604\3577201173.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ex['DepartureTime'] = ex['DepartureTime'].fillna(method='bfill')


In [8]:
VEH_RESULT

,VehicleID,Count,Volume,TravelDistance,WorkTime,TravelTime,ServiceTime,WaitingTime,TotalCost,FixedCost,VariableCost
0,VEH_2,19,305.900042,5321.622779,5009.498650,3869.498650,1140,0,4337.298223,80,4257.298223
1,VEH_3,17,265.522614,6584.812585,5680.762744,4660.762744,1020,0,5347.850068,80,5267.850068
2,VEH_22,15,465.785566,4558.067965,4124.191753,3224.191753,900,0,5619.681557,150,5469.681557
3,VEH_4,17,363.784065,6401.469598,5661.016199,4641.016199,1020,0,5201.175678,80,5121.175678
4,VEH_5,16,290.135131,6541.292271,5626.511212,4666.511212,960,0,5313.033817,80,5233.033817
...,...,...,...,...,...,...,...,...,...,...,...
471,VEH_759,1,45.000000,599.388020,453.540642,393.540642,60,0,1099.082029,200,899.082029
472,VEH_66,3,59.434316,665.569849,727.900584,547.900584,180,0,775.569849,110,665.569849
473,VEH_18,1,27.406088,214.405158,219.690762,159.690762,60,0,324.405158,110,214.405158
474,VEH_19,1,32.906627,24.521320,95.166215,35.166215,60,0,134.521320,110,24.521320


In [9]:
ORD_RESULT

,VehicleID,Sequence,ORD_NO,SiteCode,ArrivalTime,WaitingTime,ServiceTime,DepartureTime,Delivered
0,VEH_2,1,Null,O_9,2023-05-01 00:00:00,0 days 00:00:00,0,2023-05-01 00:00:00,Null
1,VEH_2,2,M18385491,D_332,2023-05-01 00:49:25.521059220,0,60,2023-05-01 01:49:25.521059220,Yes
2,VEH_2,1,Null,O_9,2023-05-01 02:41:26.338466460,0 days 03:18:33.661533540,0,2023-05-01 06:00:00,Null
3,VEH_2,2,M18387000,D_390,2023-05-01 07:00:52.671298380,0,60,2023-05-01 08:00:52.671298380,No
4,VEH_2,1,Null,O_9,2023-05-01 09:05:39.546063540,0 days 02:54:20.453936460,0,2023-05-01 12:00:00,Null
...,...,...,...,...,...,...,...,...,...
8448,VEH_19,2,MF2305086259738,D_1180,2023-05-06 12:17:29.173947960,0,60,2023-05-06 13:17:29.173947960,No
8449,VEH_19,1,Null,O_9,2023-05-06 13:35:09.972901920,NaT,0,None,Null
8450,VEH_261,1,Null,O_144,2023-05-06 18:00:00,0 days 00:00:00,0,2023-05-06 18:00:00,Null
8451,VEH_261,2,20230504B475311_001,D_68,2023-05-06 19:55:15.531713940,0,60,2023-05-06 20:55:15.531713940,No


In [10]:
sum(VEH_RESULT["Count"])

4257

In [11]:
len(df_orders_table)

0

In [12]:
VEH_RESULT.to_csv('VEH_RESULT_fitcbm.csv', index=False)
ORD_RESULT.to_csv('ORD_REUSLT_fitcbm.csv', index=False)

In [13]:
#  col = ((df_orders_table['date']==day[0]) | (df_orders_table['date']==day[1]))
#col = ((df_orders_table['date']==day[2]))
df_orders_table

,주문ID,하차지_위도,하차지_경도,착지ID,CBM,하차작업시간(분),터미널ID,date,Group,마감기한,출발시간,클러스터,Unnamed: 0,하차가능시간_종료_x,하차작업시간(분)_x,d_near_origin


In [14]:
sum(VEH_RESULT["TotalCost"])

1161774.2461302043